In [1]:
data={"intents":[
     {"tag":"greeting",
      "patterns":["Hey!","Hi there!","What’s up?","How’s it going?"],
      "responses":["Hey,what's up?","Howdy","Great to see you","Great!How can I help you?"]
     },
     { "tag":"items",
       "patterns":["Could I have a menu please?","Could I have the (dish name), please?"],
       "responses":["Ya Sure! Here are the few items:'Vada Pav','Pattis','Samosa','Manchurian'","'Vadapav','Pattis','Samosa','Manchurian'"]
     },
     {
       "tag":"famous_items", 
       "patterns":["Could you recommend the most famous item?","Could you recommend which item the people refer mostly?"],
       "responses":["The most famous item here is Vadapav","Mostly people refer Vadapav and Pattis"]
     },
     {
        "tag":"cost",
         "patterns":["Could you Please recommend the price of the items?","What are the prices of the items?"],
         "responses":["'Vadapav=18Rs','Pattis=15Rs',Samosa='20Rs','Manchurian=20Rs'","'Vadapav=18Rs','Pattis=15Rs',Samosa='20Rs','Manchurian=20Rs'"]
     },   
     {
         "tag":"goodbye",
         "patterns":["Thankyou","Thanks","Ty","Bye"],
         "responses":["Welcome!It was nice speaking to you","Welcome!Thankyou for visiting","Welcome!Next time come again","ByeBye..See you soon"]
     }
]}

In [2]:
import json
import string
import random
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [3]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to C:\Users\Akanksha
[nltk_data]     kurri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Akanksha
[nltk_data]     kurri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
lemmatizer=WordNetLemmatizer()
words=[]
classes=[]
doc_x=[]
doc_y=[]


In [5]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens=nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_x.append(pattern)
        doc_y.append(intent["tag"])
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

In [6]:
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

In [7]:
words=sorted(set(words))
classes=sorted(set(classes))

In [8]:
print(words)

['a', 'are', 'bye', 'could', 'dish', 'famous', 'going', 'have', 'hey', 'hi', 'how', 'i', 'it', 'item', 'menu', 'most', 'mostly', 'name', 'of', 'people', 'please', 'price', 'recommend', 'refer', 's', 'thanks', 'thankyou', 'the', 'there', 'ty', 'up', 'what', 'which', 'you', '’']


In [9]:
print(classes)

['cost', 'famous_items', 'goodbye', 'greeting', 'items']


In [10]:
print(doc_x)

['Hey!', 'Hi there!', 'What’s up?', 'How’s it going?', 'Could I have a menu please?', 'Could I have the (dish name), please?', 'Could you recommend the most famous item?', 'Could you recommend which item the people refer mostly?', 'Could you Please recommend the price of the items?', 'What are the prices of the items?', 'Thankyou', 'Thanks', 'Ty', 'Bye']


In [11]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'items', 'items', 'famous_items', 'famous_items', 'cost', 'cost', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [12]:
training=[]
out_empty=[0]*len(classes)

In [13]:
for idx,doc in enumerate(doc_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row=list(out_empty)
    output_row[classes.index(doc_y[idx])]=1
    training.append([bow,output_row])
random.shuffle(training)    

In [14]:
training=np.array(training,dtype=object)
train_x=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))

In [15]:
input_shape=(len(train_x[0])),
output_shape=len(train_y[0])
epochs=500

In [16]:
model=Sequential()
model.add(Dense(128,input_shape=input_shape,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape,activation='softmax'))
adam=tf.keras.optimizers.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics="accuracy")
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4608      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13,189
Trainable params: 13,189
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
model.fit(x=train_x,y=train_y,epochs=500,verbose=1)


Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 1.7301 - accuracy: 0.0714
Epoch 2/500
1/1 [==============================] - 0s 16ms/step - loss: 1.6222 - accuracy: 0.2857
Epoch 3/500
1/1 [==============================] - 0s 0s/step - loss: 1.6244 - accuracy: 0.3571
Epoch 4/500
1/1 [==============================] - 0s 0s/step - loss: 1.6871 - accuracy: 0.0714
Epoch 5/500
1/1 [==============================] - 0s 0s/step - loss: 1.6107 - accuracy: 0.1429
Epoch 6/500
1/1 [==============================] - 0s 16ms/step - loss: 1.5338 - accuracy: 0.3571
Epoch 7/500
1/1 [==============================] - 0s 16ms/step - loss: 1.4920 - accuracy: 0.3571
Epoch 8/500
1/1 [==============================] - 0s 16ms/step - loss: 1.5614 - accuracy: 0.2857
Epoch 9/500
1/1 [==============================] - 0s 16ms/step - loss: 1.5047 - accuracy: 0.3571
Epoch 10/500
1/1 [==============================] - 0s 16ms/step - loss: 1.5296 - accuracy: 0.1429
Epoch 11/500
1/1 [=========

Epoch 167/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0251 - accuracy: 1.0000
Epoch 168/500
1/1 [==============================] - 0s 0s/step - loss: 0.0319 - accuracy: 1.0000
Epoch 169/500
1/1 [==============================] - 0s 0s/step - loss: 0.0301 - accuracy: 1.0000
Epoch 170/500
1/1 [==============================] - 0s 16ms/step - loss: 0.1506 - accuracy: 0.9286
Epoch 171/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0276 - accuracy: 1.0000
Epoch 172/500
1/1 [==============================] - 0s 0s/step - loss: 0.0916 - accuracy: 1.0000
Epoch 173/500
1/1 [==============================] - 0s 0s/step - loss: 0.0901 - accuracy: 1.0000
Epoch 174/500
1/1 [==============================] - 0s 15ms/step - loss: 0.0298 - accuracy: 1.0000
Epoch 175/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0412 - accuracy: 1.0000
Epoch 176/500
1/1 [==============================] - 0s 0s/step - loss: 0.1021 - accuracy: 1.0000
Epoch 177/

1/1 [==============================] - 0s 16ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 332/500
1/1 [==============================] - 0s 0s/step - loss: 0.0195 - accuracy: 1.0000
Epoch 333/500
1/1 [==============================] - 0s 0s/step - loss: 0.0278 - accuracy: 1.0000
Epoch 334/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 335/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0392 - accuracy: 1.0000
Epoch 336/500
1/1 [==============================] - 0s 0s/step - loss: 0.0064 - accuracy: 1.0000
Epoch 337/500
1/1 [==============================] - 0s 0s/step - loss: 0.0081 - accuracy: 1.0000
Epoch 338/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 339/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0134 - accuracy: 1.0000
Epoch 340/500
1/1 [==============================] - 0s 13ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 341/500
1/1 [===

1/1 [==============================] - 0s 0s/step - loss: 0.0011 - accuracy: 1.0000
Epoch 496/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0163 - accuracy: 1.0000
Epoch 497/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 498/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 499/500
1/1 [==============================] - 0s 0s/step - loss: 0.0028 - accuracy: 1.0000
Epoch 500/500
1/1 [==============================] - 0s 0s/step - loss: 0.0044 - accuracy: 1.0000


In [18]:
def clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [19]:
def bag_of_words(text,vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx,word in enumerate(vocab):
            if word==w:
               bow[idx]=1
    return np.array(bow)            

In [20]:
def pred_class(text,vocab,labels):
    bow=bag_of_words(text,vocab)
    result=model.predict(np.array([bow]))[0]
    thresh=0.2
    y_pred=[[idx,res] for idx,res in enumerate(result) if res>thresh]
    y_pred.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list


In [21]:
def get_response(intents_list,intents_json):
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
        if i["tag"]==tag:
            result=random.choice(i["responses"])
            break
    return result        

In [ ]:
while True:
      message=input("")
      intents=pred_class(message,words,classes)  
      result=get_response(intents,data)
      print(result)  

hey
1/1 [==============================] - 0s 216ms/step
Hey,what's up?
could i have a menu please
1/1 [==============================] - 0s 47ms/step
Ya Sure! Here are the few items:'Vada Pav','Pattis','Samosa','Manchurian'
could i have the price of the items
1/1 [==============================] - 0s 43ms/step
'Vadapav=18Rs','Pattis=15Rs',Samosa='20Rs','Manchurian=20Rs'
which is the most famous item
1/1 [==============================] - 0s 47ms/step
Mostly people refer Vadapav and Pattis
thankyou
1/1 [==============================] - 0s 43ms/step
ByeBye..See you soon
hii
1/1 [==============================] - 0s 43ms/step
Welcome!Next time come again
